In [213]:
%pylab inline
import pandas as pd

Populating the interactive namespace from numpy and matplotlib


In [214]:
train_data = pd.read_csv("../CONTEST_TRAINING_SET_PUBBLICO.CSV",  
                         encoding="utf8", dtype={'COD_CAP': 'str', 'COD_ISTAT_ATECO_07':'S2' })
train_data["COD_PROVINCIA"].fillna(value="NA", inplace=True)
#train_data['COD_ISTAT_ATECO_07']=train_data['COD_ISTAT_ATECO_07'].str[0:2]

Arricchiamo il database iniziale con la decodifica della sigla provincia nel nome completo, da mostrare nella parte di visualization

In [215]:
province = pd.read_csv("../listacomuni.txt",  encoding="utf8", keep_default_na=False)

In [216]:
gr_prov_s = train_data[["COD_PROVINCIA","FLG_DEF_6M"]].groupby("COD_PROVINCIA", as_index=False)["FLG_DEF_6M"].sum()
gr_prov_s.columns = ["COD_PROVINCIA","sum"]

In [217]:
gr_prov_c = train_data[["COD_PROVINCIA","FLG_DEF_6M"]].groupby("COD_PROVINCIA", as_index=False)["FLG_DEF_6M"].count()
gr_prov_c.columns = ["COD_PROVINCIA","count"]

In [218]:
gr_prov = gr_prov_s.merge(gr_prov_c, on="COD_PROVINCIA" )

In [219]:
gr_prov["prc"] = gr_prov["sum"] / gr_prov["count"]

In [220]:
province_join = province[["Provincia", "Nome Provincia"]].drop_duplicates()

In [221]:
province_join.columns = ["COD_PROVINCIA", "Nome Provincia"]

In [222]:
export_prov = gr_prov.merge(province_join,on="COD_PROVINCIA")[["Nome Provincia","prc"]]

In [223]:
export_prov.to_csv("d3_provincia.csv", index=False, header=["provincia","prc"])

In [224]:
export_prov.head()

,Nome Provincia,prc
0,AGRIGENTO,0.242009
1,ALESSANDRIA,0.131313
2,ANCONA,0.125000
3,VALLE D'AOSTA,0.117117
4,ASCOLI PICENO,0.172414


Creiamo un file con la descrizione arricchita dei codici ATECO, considerando la categoria piu' generale rappresentata dai primi due caratteri 

In [225]:
ateco = pd.read_csv("../Transcodifica_codici_ATECO_2_cifre.csv",  
                       dtype={"ATECO2cifre":"S2", "Settore":"str" }, encoding="utf8", keep_default_na=False) 

In [226]:
gr_ateco = train_data[["COD_PROVINCIA", "FLG_DEF_6M", "COD_ISTAT_ATECO_07"]].groupby(["COD_PROVINCIA", "COD_ISTAT_ATECO_07"], as_index=False)["FLG_DEF_6M"].sum()

In [227]:
gr_ateco.columns =  ["COD_PROVINCIA", "ATECO2cifre", "sum"]
gr_ateco = gr_ateco[gr_ateco.ATECO2cifre != ""]

In [228]:
export_ateco = gr_ateco.merge(ateco,on="ATECO2cifre")[["COD_PROVINCIA", "Settore","sum"]]

In [229]:
export_ateco = export_ateco.merge(province_join,on="COD_PROVINCIA")[["Nome Provincia", "Settore","sum"]]

In [230]:
export_ateco.head()

,Nome Provincia,Settore,sum
0,AGRIGENTO,"COLTIVAZIONI AGRICOLE, PRODOTTI ANIMALI E CACCIA",1
1,AGRIGENTO,SILVICOLTURA ED UTILIZZO DI AREE FORESTALI,0
2,AGRIGENTO,INDUSTRIE ALIMENTARI,5
3,AGRIGENTO,PRODOTTI DA MINERALI NON METALLIFERI,4
4,AGRIGENTO,FABBRICAZIONE DI PRODOTTI IN METALLO (NO MACCH...,2


In [231]:
export_ateco.to_csv("d3_ateco.csv", index=False, header=["provincia","ateco","sum"])